# So sánh các phương pháp xây dựng tokenizer cho bài toán phân loại domain trên tập UIT-ViOCD

| Phương pháp tokenizer                                     | Mô tả                                                                                                                       | Ưu điểm                                                                                                                                  | Hạn chế                                                                                                                                                             | Mức độ phù hợp với UIT-ViOCD                                                                                          |
| --------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------- |
| **Tokenizer theo khoảng trắng (Whitespace-based)**        | Tách câu thành các token dựa trên dấu cách, mỗi từ được xem là một đơn vị độc lập.                                          | Triển khai đơn giản, dễ kiểm soát; phù hợp làm baseline hoặc minh họa nguyên lý cơ bản.                                                  | Không xử lý tốt lỗi chính tả, từ không dấu, teencode, emoji; từ vựng phình to; tỷ lệ OOV cao; không phản ánh cấu trúc hình thái tiếng Việt.                         | **Thấp** – chỉ phù hợp làm mô hình đối chứng (baseline), không phù hợp cho huấn luyện Transformer hiệu quả.           |
| **Tokenizer BPE tự huấn luyện trên tập dữ liệu**          | Huấn luyện Byte-Pair Encoding hoặc SentencePiece trực tiếp trên tập train của UIT-ViOCD để sinh vocab đặc thù miền dữ liệu. | Có khả năng thích nghi với dữ liệu mục tiêu; giảm OOV so với whitespace; linh hoạt về kích thước vocab.                                  | Dữ liệu huấn luyện nhỏ (~5.5k câu) dẫn đến subword kém ổn định; tokenizer dễ học theo nhiễu (lỗi chính tả, câu ngắn); chi phí triển khai cao nhưng lợi ích hạn chế. | **Trung bình** – chỉ phù hợp khi có yêu cầu bắt buộc về tokenizer tự xây dựng hoặc mục tiêu nghiên cứu tokenizer.     |
| **Tokenizer pretrained (SentencePiece / Byte-level BPE)** | Sử dụng tokenizer đã được huấn luyện trước trên corpora lớn (ví dụ PhoBERT, XLM-R), sau đó áp dụng cho dữ liệu UIT-ViOCD.   | Subword ổn định, giàu thông tin ngôn ngữ; xử lý tốt dữ liệu nhiễu, không dấu, emoji; không OOV; giúp mô hình học nhanh và tổng quát tốt. | Không chuyên biệt tuyệt đối cho miền dữ liệu; phụ thuộc vào tokenizer có sẵn.                                                                                       | **Cao** – lựa chọn phù hợp nhất cho bài toán phân loại domain với Transformer Encoder trong bối cảnh dữ liệu hạn chế. |


# 1. Đọc dữ liệu

In [2]:
import os
import sys
import json

data_dir = "./dataset/UIT-Vi-OCD"

# Đọc train/dev/test small subset data
train_data = json.load(open(os.path.join(data_dir, 'train.json'), 'r', encoding='utf-8'))
dev_data = json.load(open(os.path.join(data_dir, 'dev.json'), 'r', encoding='utf-8'))
test_data = json.load(open(os.path.join(data_dir, 'test.json'), 'r', encoding='utf-8'))

In [3]:
def summarize_split(data, name):
    print(f"{name}: {len(data):,} mẫu")

summarize_split(train_data, "Train")
summarize_split(dev_data, "Dev")
summarize_split(test_data, "Test")

print("\nSample train:")
print(train_data["0"])
print(train_data["1"])
print(train_data["2"])

Train: 4,387 mẫu
Dev: 548 mẫu
Test: 549 mẫu

Sample train:
{'review': 'gói hàng cẩn thận . chơi pubg với liên quân mượt với giá như này thì quá tốt', 'label': 'non-complaint', 'domain': 'mobile'}
{'review': 'mình góp ý thật nhé . . đừng bắt phải đăng nhập zalo nữa mình muốn tải nhạc mà không được lấy lại tôi không zalo thì gửi tin nhắn mất tiền được mã kích hoạt lại không chính xác . . mình thấy 10 ngưòi thì họ khó chịu cả 10 . . lập trình viên viết ra ứng dụng ngày càng dễ sử dụng đằng này càng ngày lại càng phức tạp và . . . bài đánh giá đầy đủ', 'label': 'complaint', 'domain': 'app'}
{'review': 'máy khá đẹp , pin trâu vân tay nhạy nhận diện khuôn mặt nhanh nói chung ổn . tuy chơi game fre fire bị chậm khung hình không mượt lắm nhưng với giá giẫm ngày 1111 được ad mã giảm giá 200k còn hơn 2tr6 thì vậy là ngon rồi', 'label'

# 2. Tokenizer

## 2.1 Lấy text trong key `review`

In [4]:
def collect_texts(data_dict):
    return [v["review"] for v in data_dict.values()]

train_texts = collect_texts(train_data)
dev_texts   = collect_texts(dev_data)
test_texts  = collect_texts(test_data)

print(f"Train: {len(train_texts)}")
print(f"Dev:   {len(dev_texts)}")
print(f"Test:  {len(test_texts)}")

print("\nSample text:")
print(train_texts[0])

Train: 4387
Dev:   548
Test:  549

Sample text:
gói hàng cẩn thận . chơi pubg với liên quân mượt với giá như này thì quá tốt


## 2.2 Chuẩn hóa text tối thiểu

Lưu ý: Tokenizer pretrained đã khá tốt, nên ta chỉ preprocessing nhẹ

In [6]:
import unicodedata

def normalize_text_nfc(text):
    text = unicodedata.normalize("NFC", text)
    text = text.strip()
    text = text.replace("\n", " ")
    return text

train_texts_nfc = [normalize_text_nfc(t) for t in train_texts]
dev_texts_nfc   = [normalize_text_nfc(t) for t in dev_texts]
test_texts_nfc  = [normalize_text_nfc(t) for t in test_texts]

In [8]:
train_texts_nfc[0]

'gói hàng cẩn thận . chơi pubg với liên quân mượt với giá như này thì quá tốt'

## 2.3 Load tokenizer pretrained

## a. XLM Roberta Tokenizer

Ưu điểm: Được train trên đa ngôn ngữ, đa domain (nhiều lĩnh vực, đã đọc nhiều từ sai chính tả, viết tắt,...), không xảy ra hiện tượng OOV (vì dùng Byte-level Byte Pair Encoding, mã hóa về byte hết nên không có trường hợp tokenizer không hiểu từ).

Khuyết điểm: Có thể tạo ra các từ vựng không có thật (subword) tuy không tốt về mặt ngôn ngữ nhưng ổn định về mặt biểu diễn.

In [11]:
!pip install transformers

  Using cached safetensors-0.7.0-cp38-abi3-win_amd64.whl.metadata (4.2 kB)
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.0 MB 7.2 MB/s eta 0:00:02
   ----------- ---------------------------- 3.4/12.0 MB 10.8 MB/s eta 0:00:01
   --------------------- ------------------ 6.6/12.0 MB 13.6 MB/s eta 0:00:01
   ------------------------------ --------- 9.2/12.0 MB 12.6 MB/s eta 0:00:01
   ---------------------------------------  11.8/12.0 MB 12.7 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 12.4 MB/s  0:00:01
Using cached safetensors-0.7.0-cp38-abi3-win_amd64.whl (341 kB)

   ------------- -------------------------- 1/3 [regex]
   -------------------------- ------------- 2/3 [transformers]
   -------------------------- ------------- 2/3 [transformers]
   -------------------------- ------------- 2/3 [transformers]
   -------------------------- ------------- 2/3 [transformers]
   -------------

In [12]:
from transformers import AutoTokenizer

TOKENIZER_NAME = "xlm-roberta-base"
xlm_tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)

print(xlm_tokenizer)

D:\app\Anaconda\envs\test\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


XLMRobertaTokenizerFast(name_or_path='xlm-roberta-base', vocab_size=250002, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	250001: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}
)


## b. PhoBert Tokenizer

Ưu điểm: Tối ưu cho tiếng Việt chuẩn

Nhược điểm: được train trên wikipedia và các nguồn văn bản chuẩn, không phù hợp cho domain review (nhiều tên code, viết tắt, sai chính tả), có thể tạo ra OOV (do dùng Sentence với BPE với đơn vị khởi tạo là unicode chứ không phải byte-level)

In [13]:
from transformers import AutoTokenizer

TOKENIZER_NAME = "vinai/phobert-base"

phobert_tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)

print(phobert_tokenizer)

PhobertTokenizer(name_or_path='vinai/phobert-base', vocab_size=64000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	64000: AddedToken("<mask>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)


In [14]:
sample_text = train_texts_nfc[0]

xlm_encoded = xlm_tokenizer(sample_text)
phobert_encoded = phobert_tokenizer(sample_text)

print("Text:")
print(sample_text)

print("\nXLM Tokens:")
print(xlm_tokenizer.convert_ids_to_tokens(xlm_encoded["input_ids"]))

print("\nPhoBERT Tokens (after NFC):")
print(phobert_tokenizer.convert_ids_to_tokens(phobert_encoded["input_ids"]))

Text:
gói hàng cẩn thận . chơi pubg với liên quân mượt với giá như này thì quá tốt

XLM Tokens:
['<s>', '▁gói', '▁hàng', '▁cẩn', '▁thận', '▁', '.', '▁chơi', '▁pub', 'g', '▁với', '▁liên', '▁quân', '▁m', 'ượt', '▁với', '▁giá', '▁như', '▁này', '▁thì', '▁quá', '▁tốt', '</s>']

PhoBERT Tokens (after NFC):
['<s>', 'gói', 'hàng', 'cẩn', 'thận', '.', 'chơi', 'pub@@', 'g', 'với', 'liên', 'quân', 'mượt', 'với', 'giá', 'như', 'này', 'thì', 'quá', 'tốt', '</s>']


Do tính đa dạng và nhiễu của dữ liệu bình luận trực tuyến, chúng tôi lựa chọn tokenizer byte-level BPE của XLM-RoBERTa nhằm đảm bảo khả năng mã hóa ổn định và tránh hiện tượng từ ngoài từ điển.

## 2.5 Hàm tokenize + padding (dùng XLM-R)

Giá trị max_len được sử dụng để cố định độ dài chuỗi token nhằm tạo batch tensor đồng nhất, xây dựng attention mask và hỗ trợ positional encoding trong Transformer, đồng thời kiểm soát chi phí tính toán

In [15]:
MAX_LEN = 128 # Thêm max len để tạo các batch (tensor hình chữ nhật) + cố định chiều cho attention mask

def tokenize_batch_xlm(texts, xlm_tokenizer, max_len):
    return xlm_tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=max_len,
        return_tensors="pt"
    )

## 2.6 Tokenize từng split (XLM-R)

Bước tokenize này chuyển mỗi câu văn bản thành chuỗi chỉ số có độ dài cố định, kèm theo attention mask để phân biệt token thật và padding, nhằm chuẩn bị đầu vào phù hợp cho Transformer Encoder

In [16]:
xlm_train_encodings = tokenize_batch_xlm(train_texts, xlm_tokenizer, MAX_LEN)
xlm_dev_encodings   = tokenize_batch_xlm(dev_texts, xlm_tokenizer, MAX_LEN)
xlm_test_encodings  = tokenize_batch_xlm(test_texts, xlm_tokenizer, MAX_LEN)

print(xlm_train_encodings.keys())
print(xlm_train_encodings["input_ids"].shape)

KeysView({'input_ids': tensor([[     0, 100929,   2508,  ...,      1,      1,      1],
        [     0,   3087,  43561,  ...,      1,      1,      1],
        [     0,   6184,  19474,  ...,      1,      1,      1],
        ...,
        [     0,      6,  11479,  ...,      1,      1,      1],
        [     0,  11521,   5791,  ...,      1,      1,      1],
        [     0,  10305,  29608,  ...,      1,      1,      1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])})
torch.Size([4387, 128])


```yaml
train_texts (list[str], length=4387)
        ↓ tokenizer
input_ids        : [4387, 128]  # token ids
attention_mask   : [4387, 128]  # 0/1 mask

## 2.7 Lấy nhãn domain

In [18]:
def collect_labels(data_dict):
    return [v["domain"] for v in data_dict.values()]

train_labels = collect_labels(train_data)
dev_labels   = collect_labels(dev_data)
test_labels  = collect_labels(test_data)

print(set(train_labels))

{'cosmetic', 'mobile', 'app', 'fashion'}


## 2.8 Mapping label

In [19]:
label2id = {label: idx for idx, label in enumerate(sorted(set(train_labels)))}
id2label = {v: k for k, v in label2id.items()}

print(label2id)

{'app': 0, 'cosmetic': 1, 'fashion': 2, 'mobile': 3}


## 2.9 Encode label thành tensor

In [20]:
import torch

def encode_labels(labels, label2id):
    return torch.tensor([label2id[label] for label in labels])

y_train = encode_labels(train_labels, label2id)
y_dev   = encode_labels(dev_labels, label2id)
y_test  = encode_labels(test_labels, label2id)

print(y_train.shape)

torch.Size([4387])


In [21]:
y_train

tensor([3, 0, 3,  ..., 0, 0, 1])

```yaml
train_texts (list[str], length=4387)
        ↓ tokenizer
input_ids        : [4387, 128]  # token ids
attention_mask   : [4387, 128]  # 0/1 mask
labels           : [4387]       # class ids

## 2.10 Sanity check

In [22]:
print("Input IDs shape:", xlm_train_encodings["input_ids"].shape)
print("Attention mask shape:", xlm_train_encodings["attention_mask"].shape)
print("Labels shape:", y_train.shape)

Input IDs shape: torch.Size([4387, 128])
Attention mask shape: torch.Size([4387, 128])
Labels shape: torch.Size([4387])


## 2.11 some check

In [23]:
idx = 0

print("Original text:")
print(train_texts[idx])

print("\nToken IDs:")
print(xlm_train_encodings["input_ids"][idx])

print("\nAttention mask:")
print(xlm_train_encodings["attention_mask"][idx])

print("\nTokens:")
print(xlm_tokenizer.convert_ids_to_tokens(
    xlm_train_encodings["input_ids"][idx]
))

Original text:
gói hàng cẩn thận . chơi pubg với liên quân mượt với giá như này thì quá tốt

Token IDs:
tensor([     0, 100929,   2508,  24376,   5675,  12976,     19,      6,      5,
         19702,  64792,    177, 137322,     14,   8151,  29225,    347,  11479,
         12435,     18, 137322,     14,   3816,   1641,   1617,   2579,   6526,
         29163,     18,      2,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,  

In [24]:
pad_id = xlm_tokenizer.pad_token_id

num_pad = (xlm_train_encodings["input_ids"] == pad_id).sum(dim=1)

print("Số padding token (5 câu đầu):")
print(num_pad[:5])

Số padding token (5 câu đầu):
tensor([ 98,   7,  52,  75, 109])


# 3. Lưu file pt

In [25]:
import torch
import os

processed_dir = "./dataset/UIT-Vi-OCD/processed"
os.makedirs(processed_dir, exist_ok=True)

def save_split(filename, encodings, labels):
    path = os.path.join(processed_dir, filename)
    torch.save(
        {
            "input_ids": encodings["input_ids"],
            "attention_mask": encodings["attention_mask"],
            "labels": labels
        },
        path
    )

save_split("train.pt", xlm_train_encodings, y_train)
save_split("dev.pt",   xlm_dev_encodings,   y_dev)
save_split("test.pt",  xlm_test_encodings,  y_test)

In [26]:
train_data = torch.load(
    "./dataset/UIT-Vi-OCD/processed/train.pt"
)

print(train_data.keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


# 4. Post-tokenization EDA

In [27]:
train_data.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [28]:
# Phân bố độ dài thật (không tính padding)

input_ids = train_data["input_ids"]
attention_mask = train_data["attention_mask"]

real_lengths = attention_mask.sum(dim=1)

print(
    real_lengths.min().item(),
    real_lengths.float().mean().item(),
    real_lengths.max().item()
)

3 45.069068908691406 128


Min = 3 token | Max = 128 token | Mean = 45 token

In [29]:
# Tỷ lệ padding

pad_ratio = 1 - real_lengths.float() / input_ids.size(1)
print(pad_ratio.mean())

tensor(0.6479)


In [30]:
# Phân bố nhãn
labels = train_data["labels"]

unique, counts = torch.unique(labels, return_counts=True)
print(dict(zip(unique.tolist(), counts.tolist())))

{0: 1613, 1: 1043, 2: 1368, 3: 363}
